In [ ]:
import pandas as pd
import json
from tqdm import tqdm
from os import walk

from Stats_utils import *
from langdetect import detect
import numpy as np

from scipy import stats
from scipy.stats import pearsonr
from scipy.stats import spearmanr

## RQ 2 -- LLMs relations to cultural distances

In [3]:
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

In [4]:
def count_tokens(recette, thrs=20):
    list_recette = recette.split()
    if len(list_recette) > thrs:
        return True
    else:
        return False

In [ ]:
ROOT_DIR = '.'
my_path = f"./LLM_scored/config_run/"
filenames = list(set(next(walk(my_path), (None,None,[]))[2]))

495


In [ ]:
## LOADING CULTURAL DISTANCES DATASETS
df_ingle = pd.read_excel(f'./CulturalDistanceEVSWVS.xlsx')
df_ingle = df_ingle.dropna(subset=['Country'])
dist_ingle = inglehart_dist(df_ingle)

df_geo = pd.read_excel(f'./Country_latLong.xlsx')
df_geo = df_geo.dropna(subset=['Country'])
dist_geo = geograph_dist(df_geo)

dist_ling = pd.read_excel(f'./Lang-Dist_all-years_n22350.xlsx')
dist_rel = pd.read_csv(f'./religious_data.csv')

/home/mila/f/florian.carichon/.conda/envs/work0/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [ ]:
#Changer for the propoer analyzed layers
layer_list = ['layer_original', 'layer_1', 'layer_23', 'layer_-1', 'layer_last']

## Measuring Cultural correaltions

In [ ]:
# Initializing all metrics we want to save for correlation and logistic regression analyses
all_new_div, all_new_prob, all_new_extr, all_new_rank = [], [], [], []
all_uniq_dist, all_uniq_proto, all_diff_glob, all_diff_loc = [], [], [], []
all_nsurp, all_dsurp = [], []
all_dist_ingle, all_dist_ling, all_dist_geo, all_dist_rel = [], [], [], []
nb_ingr, nb_new_ingr, ratio_new_ingr, clean_text_length, clean_nb_uniq_tokens, lexical_div, ratio_length = [], [], [], [], [], [], []

#FOR LINGUISTIC OR RELIGIOUS DISTANCE
country_list_1 = list(set(dist_ling['country_i']))
country_list_2 = list(set(dist_rel['country_i']))
#FOR INGLEHART CULTURAl or GEOGRAPHICAL DISTANCE
country_list_3 = list(df_geo['Country'])
country_list_4 = list(df_ingle['Country'])

for k in tqdm(range(len(filenames))):
    filename = filenames[k]
    file_path = my_path + filename
    with open(file_path) as json_file:
        recipe_dict1 = json.load(json_file)

    
    country = recipe_dict1['Country']
    ##Exceptions due to the Congo Bars recipes
    if country == 'congo':
        country = 'united states'

    ### Uncomment here for linguistic and relgion analyses
    # if country in country_list_1:
    #     country_dist1 = dist_ling[dist_ling['country_i'] == country]
    #     country_dist2 = dist_rel[dist_rel['country_i'] == country]
        #print(country_dist2)
    ## Uncomment here for cultural and geographical distances
    if country in country_list_3 and country in country_list_4:
        country_dist3 = dist_geo[country]
        country_dist4 = dist_ingle[country]

        variation_list = list(recipe_dict1['LLM_gen']['variation_novelty'].keys())
        varia_country_list = [key.split("_")[1] for key in variation_list]

        for i, variation in enumerate(variation_list):
            varia_country = varia_country_list[i]
            if varia_country == 'taiwan': ## For geo only
                varia_country = 'japan' 
            ### Uncomment here for linguistic and relgion analyses
            # if varia_country in country_list_1:
            if varia_country in country_list_3 and varia_country in country_list_4:
                ingle_dist = get_distance_LLMs(varia_country, country_dist3, type='ingle')
                geo_dist = get_distance_LLMs(varia_country, country_dist4, type='geo')
                ### Uncomment here for linguistic and relgion analyses
                # ling_dist = get_distance_LLMs(varia_country, country_dist1)
                # reli_dist = get_distance_LLMs(varia_country, country_dist2, type='reli')
                list_recipe_var = list(recipe_dict1['LLM_gen']['variation_novelty'][variation].keys())
                for recipe in list_recipe_var:
                    is_eng = is_english(recipe_dict1['LLM_gen']['variation_novelty'][variation][recipe]['Instructions'])
                    if is_eng:
                        count_ = count_tokens(recipe_dict1['LLM_gen']['variation_novelty'][variation][recipe]['Instructions'])
                        if count_:
                            all_dist_ingle.append(ingle_dist)
                            all_dist_geo.append(geo_dist)
                            ### Uncomment here for linguistic and relgion analyses
                            # all_dist_ling.append(ling_dist)
                            # all_dist_rel.append(reli_dist)
                            all_new_div.append(recipe_dict1['LLM_gen']['variation_novelty'][variation][recipe]['layer_original']['newness_div'])
                            all_uniq_dist.append(recipe_dict1['LLM_gen']['variation_novelty'][variation][recipe]['layer_original']['uniq_dist'])
                            all_diff_loc.append(recipe_dict1['LLM_gen']['variation_novelty'][variation][recipe]['layer_original']['diff_local'])
                            all_nsurp.append(recipe_dict1['LLM_gen']['variation_novelty'][variation][recipe]['layer_original']['new_surprise'])
                            all_dsurp.append(recipe_dict1['LLM_gen']['variation_novelty'][variation][recipe]['layer_original']['dist_surprise'])

data = {
    'newness_div': all_new_div,
    'uniq_dist': all_uniq_dist,
    'diff_local': all_diff_loc,
    'new_surprise': all_nsurp,
    'dist_surprise': all_dsurp,
    'dist_ingle': all_dist_ingle,
    #'dist_ling': all_dist_ling,
    'dist_geo': all_dist_geo,
}

# Filter out keys where the list is empty
filtered_data = {k: v for k, v in data.items() if len(v) > 0}

100%|██████████| 495/495 [16:48<00:00,  2.04s/it]


In [ ]:
#Control purpose
print(len(all_new_div), len(all_dist_ling), len(all_dist_rel))

148474 0 0


In [ ]:
#FOR CULTURAL AND GEOGRAPHICAL ONLY (SEE BELOW FOR LINGUISTIC)
correlation_new, p_value_new = pearsonr(all_dist_ingle, all_new_div)
correlation_uniq, p_value_uniq = pearsonr(all_dist_ingle, all_uniq_dist)
correlation_diff, p_value_diff = pearsonr(all_dist_ingle, all_diff_loc)
correlation_nsurp, p_value_nsurp = pearsonr(all_dist_ingle, all_nsurp)
correlation_dsurp, p_value_dsurp = pearsonr(all_dist_ingle, all_dsurp)

### Uncomment here for Spearman correlations
# correlation_new, p_value_new = spearmanr(all_dist_ingle, all_new_div)
# correlation_uniq, p_value_uniq = spearmanr(all_dist_ingle, all_uniq_dist)
# correlation_diff, p_value_diff = spearmanr(all_dist_ingle, all_diff_loc)
# correlation_nsurp, p_value_nsurp = spearmanr(all_dist_ingle, all_nsurp)
# correlation_dsurp, p_value_dsurp = spearmanr(all_dist_ingle, all_dsurp)

print('NEW : ', round(correlation_new,3), '(',round(p_value_new, 3),')')
print('UNIQ : ', round(correlation_uniq,3), '(',round(p_value_uniq, 3),')')
print('DIFF : ', round(correlation_diff,3), '(',round(p_value_diff, 3),')')
print('NSURP : ', round(correlation_nsurp,3), '(',round(p_value_nsurp, 3),')')
print('DSURP : ', round(correlation_dsurp,3), '(',round(p_value_dsurp, 3),')')
print('========================================================================')


correlation_new, p_value_new = pearsonr(all_dist_geo, all_new_div)
correlation_uniq, p_value_uniq = pearsonr(all_dist_geo, all_uniq_dist)
correlation_diff, p_value_diff = pearsonr(all_dist_geo, all_diff_loc)
correlation_nsurp, p_value_nsurp = pearsonr(all_dist_geo, all_nsurp)
correlation_dsurp, p_value_dsurp = pearsonr(all_dist_geo, all_dsurp)

### Uncomment here for Spearman correlations
# correlation_new, p_value_new = spearmanr(all_dist_geo, all_new_div)
# correlation_uniq, p_value_uniq = spearmanr(all_dist_geo, all_uniq_dist)
# correlation_diff, p_value_diff = spearmanr(all_dist_geo, all_diff_loc)
# correlation_nsurp, p_value_nsurp = spearmanr(all_dist_geo, all_nsurp)
# correlation_dsurp, p_value_dsurp = spearmanr(all_dist_geo, all_dsurp)

print('NEW : ', round(correlation_new,3), '(',round(p_value_new, 3),')')
print('UNIQ : ', round(correlation_uniq,3), '(',round(p_value_uniq, 3),')')
print('DIFF : ', round(correlation_diff,3), '(',round(p_value_diff, 3),')')
print('BSURP : ', round(correlation_nsurp,3), '(',round(p_value_nsurp, 3),')')
print('DSURP : ', round(correlation_dsurp,3), '(',round(p_value_dsurp, 3),')')
print('========================================================================')


NEW :  0.004 ( 0.09 )
UNIQ :  -0.031 ( 0.0 )
DIFF :  -0.009 ( 0.0 )
NSURP :  -0.029 ( 0.0 )
DSURP :  0.029 ( 0.0 )
NEW :  -0.029 ( 0.0 )
UNIQ :  0.021 ( 0.0 )
DIFF :  -0.004 ( 0.11 )
BSURP :  0.008 ( 0.001 )
DSURP :  0.004 ( 0.13 )


In [ ]:
#FOR LINGUISITIC AND RELIGIOUS

correlation_new, p_value_new = pearsonr(all_dist_ling, all_new_div)
correlation_uniq, p_value_uniq = pearsonr(all_dist_ling, all_uniq_dist)
correlation_diff, p_value_diff = pearsonr(all_dist_ling, all_diff_loc)
correlation_nsurp, p_value_nsurp = pearsonr(all_dist_ling, all_nsurp)
correlation_dsurp, p_value_dsurp = pearsonr(all_dist_ling, all_dsurp)

# correlation_new, p_value_new = spearmanr(all_dist_ling, all_new_div)
# correlation_uniq, p_value_uniq = spearmanr(all_dist_ling, all_uniq_dist)
# correlation_diff, p_value_diff = spearmanr(all_dist_ling, all_diff_loc)
# correlation_nsurp, p_value_nsurp = spearmanr(all_dist_ling, all_nsurp)
# correlation_dsurp, p_value_dsurp = spearmanr(all_dist_ling, all_dsurp)

print('NEW : ', round(correlation_new,3), '(',round(p_value_new, 3),')')
print('UNIQ : ', round(correlation_uniq,3), '(',round(p_value_uniq, 3),')')
print('DIFF : ', round(correlation_diff,3), '(',round(p_value_diff, 3),')')
print('NSURP : ', round(correlation_nsurp,3), '(',round(p_value_nsurp, 3),')')
print('DSURP : ', round(correlation_dsurp,3), '(',round(p_value_dsurp, 3),')')
print('========================================================================')


correlation_new, p_value_new = pearsonr(all_dist_rel, all_new_div)
correlation_uniq, p_value_uniq = pearsonr(all_dist_rel, all_uniq_dist)
correlation_diff, p_value_diff = pearsonr(all_dist_rel, all_diff_loc)
correlation_nsurp, p_value_nsurp = pearsonr(all_dist_rel, all_nsurp)
correlation_dsurp, p_value_dsurp = pearsonr(all_dist_rel, all_dsurp)

# correlation_new, p_value_new = spearmanr(all_dist_rel, all_new_div)
# correlation_uniq, p_value_uniq = spearmanr(all_dist_rel, all_uniq_dist)
# correlation_diff, p_value_diff = spearmanr(all_dist_rel, all_diff_loc)
# correlation_nsurp, p_value_nsurp = spearmanr(all_dist_rel, all_nsurp)
# correlation_dsurp, p_value_dsurp = spearmanr(all_dist_rel, all_dsurp)

print('NEW : ', round(correlation_new,3), '(',round(p_value_new, 3),')')
print('UNIQ : ', round(correlation_uniq,3), '(',round(p_value_uniq, 3),')')
print('DIFF : ', round(correlation_diff,3), '(',round(p_value_diff, 3),')')
print('BSURP : ', round(correlation_nsurp,3), '(',round(p_value_nsurp, 3),')')
print('DSURP : ', round(correlation_dsurp,3), '(',round(p_value_dsurp, 3),')')
print('========================================================================')

NEW :  -0.06 ( 0.0 )
UNIQ :  0.018 ( 0.0 )
DIFF :  -0.014 ( 0.0 )
NSURP :  -0.008 ( 0.001 )
DSURP :  -0.001 ( 0.765 )
NEW :  -0.01 ( 0.0 )
UNIQ :  -0.017 ( 0.0 )
DIFF :  -0.009 ( 0.0 )
BSURP :  -0.009 ( 0.0 )
DSURP :  0.046 ( 0.0 )
